# Applying VADER for Sentence-Level Sentiment Detection in Social Media Text

## Code implementation using 

In [ ]:
!pip install transformers
!pip install simpletransformers
!pip install vaderSentiment
!pip install scikit-learn

In [10]:
import spacy
from transformers import pipeline 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datasets import load_dataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

nlp = spacy.load("en_core_web_sm")
vader = SentimentIntensityAnalyzer()

In [ ]:
#loading the dataset from Hugging Face's datasets
dataset = load_dataset("mteb/tweet_sentiment_extraction")
#Converting into vectors 
train_dataset = dataset['train'].to_pandas()

In [ ]:
def vader_output_to_label(vader_output):
    compound = vader_output['compound']
    if compound < 0:
        return 'negative'
    elif compound == 0.0:
        return 'neutral'
    else:
        return 'positive'

def run_vader_spacy(textual_unit, lemmatize=False, pos_to_include=None, verbose=0):
    doc = nlp(textual_unit)
    input_tokens = []

    for token in doc:
        if pos_to_include is None or token.pos_ in pos_to_include:
            word = token.lemma_ if lemmatize else token.text
            input_tokens.append(word)

    processed_text = " ".join(input_tokens)
    scores = vader.polarity_scores(processed_text)

    if verbose:
        print("Input to VADER:", processed_text)
        print("Scores:", scores)

    return scores    

#Matching the dataset's labels with their corresponding Sentiments 
label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
train_dataset['gold_label'] = train_dataset['label'].map(label_map)

In [ ]:
#Applying VADER method ot each tweet in the dataset 
def get_vader_label(row):
    scores = run_vader_spacy(row['text'], lemmatize=True)
    return vader_output_to_label(scores)


#Applying VADER to each row in the dataset 
train_dataset['vader_label'] = train_dataset.apply(get_vader_label, axis=1)

#Saving the file for qualitative and quantitative analysis
train_dataset[['text', 'gold_label', 'vader_label']].to_csv("vader_sentiment_analysis.csv", index=False)

## Quantitative Analysis 

In [13]:
# Compare VADER predictions to gold labels
y_true = train_dataset['gold_label']
y_pred = train_dataset['vader_label']

# Overall metrics
print("Classification Report:")
print(classification_report(y_true, y_pred))


# Accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Classification Report:
              precision    recall  f1-score   support

    negative       0.70      0.63      0.66      7781
     neutral       0.74      0.43      0.54     11118
    positive       0.55      0.89      0.68      8582

    accuracy                           0.63     27481
   macro avg       0.66      0.65      0.63     27481
weighted avg       0.67      0.63      0.62     27481

Accuracy: 0.6311
